# Lab : Image Classification using Convolutional Neural Networks

At the end of this laboratory, you would get familiarized with

*   Creating deep networks using Keras
*   Steps necessary in training a neural network
*   Prediction and performance analysis using neural networks

---

# **In case you use a colaboratory environment**
By default, Colab notebooks run on CPU.
You can switch your notebook to run with GPU.

In order to obtain access to the GPU, you need to choose the tab Runtime and then select “Change runtime type” as shown in the following figure:

![Changing runtime](https://miro.medium.com/max/747/1*euE7nGZ0uJQcgvkpgvkoQg.png)

When a pop-up window appears select GPU. Ensure “Hardware accelerator” is set to GPU.

# **Working with a new dataset: CIFAR-10**

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. More information about CIFAR-10 can be found [here](https://www.cs.toronto.edu/~kriz/cifar.html).

In Keras, the CIFAR-10 dataset is also preloaded in the form of four Numpy arrays. x_train and y_train contain the training set, while x_test and y_test contain the test data. The images are encoded as Numpy arrays and their corresponding labels ranging from 0 to 9.

Your task is to:

*   Visualize the images in CIFAR-10 dataset. Create a 10 x 10 plot showing 10 random samples from each class.
*   Convert the labels to one-hot encoded form.
*   Normalize the images.




In [1]:
# Set the number of threads based on available CPU cores (if needed)

# import tensorflow as tf
# tf.config.threading.set_inter_op_parallelism_threads(6) # 8
# tf.config.threading.set_intra_op_parallelism_threads(14) # 24

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Plotting 10 random pictures from each class

plt.figure(figsize=(12, 12))
for label in range(10):
    indices = np.where(y_train == label)[0]
    random_pics = np.random.choice(indices, 10, replace = False)
    for i, pic in enumerate(random_pics):
        plt.subplot(10, 10, label * 10 + i + 1)
        plt.imshow(x_train[pic])
        plt.axis('off')
plt.show()

In [ ]:
# One-hot encoding the labels

y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

print('y_train[0]:', y_train[0])
print('y_test[-1]:', y_test[-1])

In [ ]:
# Normalizing the images

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print('x_train.min():', x_train.min())
print('x_train.max():', x_train.max())

## Define the following model (same as the one in tutorial)

For the convolutional front-end, start with a single convolutional layer with a small filter size (3,3) and a modest number of filters (32) followed by a max pooling layer. 

Use the input as (32,32,3). 

The filter maps can then be flattened to provide features to the classifier. 

Use a dense layer with 100 units before the classification layer (which is also a dense layer with softmax activation).

In [ ]:
from keras.backend import clear_session

clear_session()

In [ ]:
# Defining the model

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dense(10, activation = 'softmax'))

model.summary()

*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 50 epochs with a batch size of 512.

In [ ]:
# Compiling the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

# Training the model
history = model.fit(x_train, y_train, batch_size = 512, epochs = 50, validation_split = 0.1)

*   Plot the cross entropy loss curve and the accuracy curve

In [ ]:
print(history.history.keys())

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color = 'blue', label = 'train')
plt.plot(history.history['val_loss'], color = 'red', label = 'val')

plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color = 'green', label = 'train')
plt.plot(history.history['val_accuracy'], color = 'red', label = 'val')

plt.show()

## Defining Deeper Architectures: VGG Models

*   Define a deeper model architecture for CIFAR-10 dataset and train the new model for 50 epochs with a batch size of 512. We will use VGG model as the architecture.

Stack two convolutional layers with 32 filters, each of 3 x 3. 

Use a max pooling layer and next flatten the output of the previous layer and add a dense layer with 128 units before the classification layer. 

For all the layers, use ReLU activation function. 

Use same padding for the layers to ensure that the height and width of each layer output matches the input


In [10]:
clear_session()

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same', input_shape = (32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dense(10, activation = 'softmax'))

*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 50 epochs with a batch size of 512.

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

history_2 = model.fit(x_train, y_train, batch_size = 512, epochs = 50, validation_split = 0.1)

*   Compare the performance of both the models by plotting the loss and accuracy curves of both the training steps. Does the deeper model perform better? Comment on the observation.
 

In [ ]:
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color = 'blue', label = 'model 1')
plt.plot(history_2.history['loss'], color = 'red', label = 'model 2')

plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color = 'blue', label = 'model 1')
plt.plot(history_2.history['accuracy'], color = 'red', label = 'model 2')

plt.show()

**Comment on the observation**

The deeper model leads to lower loss and higher accuracy.  
It simply has more layers, which helps it learn more complex patterns in the data.

*   Use predict function to predict the output for the test split
*   Plot the confusion matrix for the new model and comment on the class confusions.


In [ ]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(x_test)

predictions = np.argmax(predictions, axis = 1)
gt = np.argmax(y_test, axis = 1)

confusion_matrix(gt, predictions)

**Comment here :**

The model performs well on classes 6 and 7, but struggles with others.  
E.g. class 2 is often confused with class 4, and class 5 with class 3.

*    Print the test accuracy for the trained model.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

## Define the complete VGG architecture.

Stack two convolutional layers with 64 filters, each of 3 x 3 followed by max pooling layer. 

Stack two more convolutional layers with 128 filters, each of 3 x 3, followed by max pooling, followed by two more convolutional layers with 256 filters, each of 3 x 3, followed by max pooling. 

Flatten the output of the previous layer and add a dense layer with 128 units before the classification layer. 

For all the layers, use ReLU activation function. 

Use same padding for the layers to ensure that the height and width of each layer output matches the input

*   Change the size of input to 64 x 64.

In [16]:
clear_session()

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same', input_shape = (64, 64, 3)))
model.add(Conv2D(64, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
model.add(Conv2D(128, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
model.add(Conv2D(256, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', padding = 'same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dense(10, activation = 'softmax'))

*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 10 epochs with a batch size of 512.
*   Predict the output for the test split and plot the confusion matrix for the new model and comment on the class confusions.

In [18]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [19]:
# Resizing images to 64x64
x_train = tf.image.resize(x_train, (64, 64))
x_test = tf.image.resize(x_test, (64, 64))

In [ ]:
history = model.fit(x_train, y_train, batch_size = 512, epochs = 10)

In [ ]:
predictions = model.predict(x_test)

predictions = np.argmax(predictions, axis = 1)

confusion_matrix(gt, predictions)

In [24]:
# The new model generally improves accuracy for classes 0, 4, 5, and 9.
# But it still struggles with class 2 and is confusing similar classes (3/5 and 1/9).

# Understanding deep networks

*   What is the use of activation functions in network? Why is it needed?
*   We have used softmax activation function in the exercise. There are other activation functions available too. What is the difference between sigmoid activation and softmax activation?
*   What is the difference between categorical crossentropy and binary crossentropy loss?

**Write the answers below :**

* 1 - Use of activation functions:

Activation functions give a network the ability to learn complex patterns by introducing non-linearity.  
Without them, the network would only be able to model simple linear relationships.

* 2 - Key Differences between sigmoid and softmax:

Softmax is used for multi-class classification, as it converts outputs into probabilities for multiple classes.  
Sigmoid is used for binary classification, giving a probability between 0 and 1 for a single class.

* 3 - Key Differences between categorical crossentropy and binary crossentropy loss:

Categorical crossentropy is used when predicting more than two classes, comparing probabilities for each class.  
Binary crossentropy is used for two-class problems, comparing probabilities for just two outcomes (0 or 1).